In [ ]:
!pip install darts==0.16.1
from typing import Tuple, Sequence, Optional, Union, Dict
import math
from abc import ABC
import torch
import torch.nn.functional as F
from torch import nn
from torch.nn.utils import weight_norm
from torch import Tensor
from darts import TimeSeries
from darts.logging import raise_if_not, get_logger
from darts.utils.likelihood_models import Likelihood
from darts.utils.torch import random_method
from darts.utils.likelihood_models import QuantileRegression
from darts.dataprocessing.transformers import Scaler
from darts.models import TFTModel
import numpy as np
from numpy.random.mtrand import RandomState
import pandas as pd
import matplotlib.pyplot as plt

from darts_utility import SolarFlare, SolarFlare_No_Attention, SolarFlare_No_Bigru

logger = get_logger(__name__)

In [ ]:
path = '../input/irradiance-dataset/irradiance_dataset.csv'
df = pd.read_csv(path)[:60000]
df = df.drop(columns="Unnamed: 0")
memory = 24*6
horizon = 5
batch = 256

df.insert(0, 'GHI', df.pop('GHI'))

# Split data
target_ts = TimeSeries.from_series(df["GHI"]).astype(np.float32)
target_train, target_val = target_ts.split_after(0.8)
target_val, target_test = target_val.split_after(0.5)


past_covar_ts = TimeSeries.from_dataframe(df[df.columns.to_list()[1:]]).astype(np.float32)
past_covar_train, past_covar_val = past_covar_ts.split_after(0.8)
past_covar_val, past_covar_test = past_covar_val.split_after(0.5)

# Scale data
target_ts_scaler = Scaler()
target_train = target_ts_scaler.fit_transform(target_train)
target_val = target_ts_scaler.transform(target_val)
target_test = target_ts_scaler.transform(target_test)
series_transformed = target_ts_scaler.transform(target_ts)

covar_ts_scaler = Scaler()
covar_train = covar_ts_scaler.fit_transform(past_covar_train)
covar_val = covar_ts_scaler.transform(past_covar_val)
covar_test = covar_ts_scaler.transform(past_covar_test)


In [ ]:
model1 = SolarFlare(input_chunk_length=memory,
                   kernel_size=2,
                   dilation_base=2,
                   output_chunk_length=horizon,
                   likelihood=QuantileRegression(quantiles=[0.01, 0.05, 0.2, 0.5, 0.8, 0.95, 0.99]),
                   batch_size=batch,
                   hidden_size=128,
                   nr_epochs_val_period=1,
                   log_tensorboard=False,
                   torch_device_str= "cuda:0")

model2 = SolarFlare_No_Attention(input_chunk_length=memory,
                   kernel_size=2,
                   dilation_base=2,
                   output_chunk_length=horizon,
                   likelihood=QuantileRegression(quantiles=[0.01, 0.05, 0.2, 0.5, 0.8, 0.95, 0.99]),
                   batch_size=batch,
                   hidden_size=128,
                   nr_epochs_val_period=1,
                   log_tensorboard=False,
                   torch_device_str= "cuda:0")

model3 = SolarFlare_No_Bigru(input_chunk_length=memory,
                   kernel_size=2,
                   dilation_base=2,
                   output_chunk_length=horizon,
                   likelihood=QuantileRegression(quantiles=[0.01, 0.05, 0.2, 0.5, 0.8, 0.95, 0.99]),
                   batch_size=batch,
                   hidden_size=128,
                   nr_epochs_val_period=1,
                   log_tensorboard=False,
                   torch_device_str= "cuda:0")

model4 = TFTModel(input_chunk_length=memory,
                output_chunk_length=horizon,
                hidden_size=128,
                lstm_layers=1,
                num_attention_heads=4,
                dropout=0.2,
                likelihood=QuantileRegression(quantiles=[0.01, 0.05, 0.2, 0.5, 0.8, 0.95, 0.99]),
                batch_size=batch,
                nr_epochs_val_period=1,
                torch_device_str="cuda:0")

In [ ]:
model1.fit(series=target_train, past_covariates=covar_train, val_series=target_val, val_past_covariates=covar_val, verbose=True, num_loader_workers=2)
model1.save_model("SolarFlare60k_bigru_attention.pth.tar")
plt.figure(figsize=(30, 7))
backcast1 = model1.historical_forecasts(series=target_train, past_covariates=past_covar_train, num_samples=600, start=0.98, retrain=False, verbose=True)
backcast1.quantile_df(quantile=0.5).to_csv("backcast1_60k_bigru_attention_median.csv")
series_transformed.slice_intersect(backcast1).plot(label="target")
backcast1.backcast.plot(label="forecast", low_quantile=0.01, high_quantile=0.99)
plt.xlabel("timesteps")
plt.ylabel("GHI")
plt.title("Backcast of model with attention and bigru")
plt.legend()
plt.save("backcast1_60k_bigru_attention.png")
plt.show()

In [ ]:
model2.fit(series=target_train, past_covariates=covar_train, val_series=target_val, val_past_covariates=covar_val, verbose=True, num_loader_workers=2)
model2.save_model("SolarFlare30k_no_attention.pth.tar")
plt.figure(figsize=(30, 7))
backcast2 = model2.historical_forecasts(series=target_train, past_covariates=past_covar_train, num_samples=300, start=0.97, retrain=False, verbose=True)
backcast2.quantile_df(quantile=0.5).to_csv("backcast2_30k_no_attention.csv")
series_transformed.slice_intersect(backcast2).plot(label="target")
backcast2.plot(label="forecast", low_quantile=0.01, high_quantile=0.99)
plt.xlabel("timesteps")
plt.ylabel("GHI")
plt.title("Backcast of model with no attention")
plt.legend()
plt.savefig("backcast2_30k_no_attention.svg", format="svg")
plt.show()


In [ ]:
model3.fit(series=target_train, past_covariates=covar_train, val_series=target_val, val_past_covariates=covar_val, verbose=True, num_loader_workers=2)
model3.save_model("SolarFlare30k_no_bigru.pth.tar")
backcast3 = model3.historical_forecasts(series=target_train, past_covariates=past_covar_train, num_samples=300, start=0.97, retrain=False, verbose=True)
backcast3.quantile_df(quantile=0.5).to_csv("backcast3_30k_no_bigru.csv")
plt.figure(figsize=(20, 20))
series_transformed.slice_intersect(backcast3).plot(label="target")
backcast3.backcast.plot(label="forecast", low_quantile=0.01, high_quantile=0.99)
plt.xlabel("timesteps")
plt.ylabel("GHI")
plt.title("Backcast of model with no bidirectional gru")
plt.legend()
plt.savefig("backcast3_30k_no_bigru.svg", format="svg")
plt.show()

In [ ]:
plt.figure(figsize=(20, 7))
series_transformed.slice_intersect(backcast1).plot(label="target")
backcast1.plot(label="forecast", low_quantile=0.01, high_quantile=0.99)
plt.xlabel("timesteps")
plt.ylabel("GHI")
plt.title("Backcast of model with attention and bigru")
plt.legend()
plt.savefig("backcast1_60k_bigru_attention.svg", format="svg")
plt.show()
